In [15]:
import numpy as np

In [16]:
d = np.array([0.1807, 0, 0, 0.17415, 0.11985, 0.11655])
r = np.array([0, -0.6127, -0.57155, 0, 0, 0])
alpha = np.array([np.pi/2, 0, 0, np.pi/2, -np.pi/2, 0])

In [17]:
def T(d, theta, r, alpha):
    return np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), r*np.cos(theta)],
        [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), r*np.sin(theta)],
        [0, np.sin(alpha), np.cos(alpha), d],
        [0, 0, 0, 1]])

def W(omega, v):
    return np.array([
        [0, -omega[2], omega[1], v[0]],
        [omega[2], 0, -omega[0], v[1]],
        [-omega[1], omega[0], 0, v[2]],
        [0, 0, 0]])

In [29]:
p = np.eye(4)
for i in range(len(d)):
    p = p@T(d[i], 0, r[i], alpha[i])
p@np.array([0, 0, 0, 1])

array([-1.18425, -0.2907 ,  0.06085,  1.     ])